In [ ]:
import os
import rasterio
import numpy as np
import torch
from torchvision import transforms
from PIL import Image
import pandas as pd
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
import openmeteo_requests
import requests_cache
from retry_requests import retry
from datetime import datetime

# Device configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Image transformation for CNN input
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

# Load trained CNN model
model = torch.load("cnn_model.pth", map_location=DEVICE)
model.eval()

def extract_wvr_from_geotiff(geotiff_path):
    with rasterio.open(geotiff_path) as src:
        image = src.read([1, 2, 3])  # Assuming RGB channels
        image = np.transpose(image, (1, 2, 0))  # Convert to (H, W, C)
        image = Image.fromarray(image.astype('uint8'))
        image = transform(image).unsqueeze(0).to(DEVICE)
        
        with torch.no_grad():
            wvr = model(image).item()
    
    return wvr

# Load trained Ridge regression models
features = ['WVR', 'CHLA_RESULT', 'PIP_PT', 'TIP_PT', 'EVI']
targets = ['AMMONIA_N_RESULT', 'CHLORIDE_RESULT', 'COND_RESULT', 'DOC_RESULT',
           'NITRATE_NITRITE_N_RESULT', 'NTL_RESULT', 'PH_RESULT', 'PTL_RESULT',
           'SULFATE_RESULT', 'TKN_RESULT', 'TURB_RESULT']

ridge_models = {}
for target in targets:
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('poly', PolynomialFeatures(degree=3, include_bias=False)),
        ('model', Ridge(alpha=1.0))
    ])
    pipeline.named_steps['model'].coef_ = np.load(f"{target}_coef.npy")  # Load model coefficients
    pipeline.named_steps['model'].intercept_ = np.load(f"{target}_intercept.npy")
    ridge_models[target] = pipeline

# Setup Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

def fetch_weather_data(lat, lon):
    current_date = datetime.now().strftime("%Y-%m-%d")
    
    # Fetch daily weather data
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": lat,
        "longitude": lon,
        "daily": ["temperature_2m_max", "temperature_2m_min", "precipitation_sum"]
    }
    responses = openmeteo.weather_api(url, params=params)
    response = responses[0]
    daily = response.Daily()
    
    temp_max = daily.Variables(0).ValuesAsNumpy()[0]
    temp_min = daily.Variables(1).ValuesAsNumpy()[0]
    precipitation = daily.Variables(2).ValuesAsNumpy()[0]
    
    # Fetch climate change model data
    climate_url = "https://climate-api.open-meteo.com/v1/climate"
    params_climate = {
        "latitude": lat,
        "longitude": lon,
        "start_date": current_date,
        "end_date": current_date,
        "models": ["CMCC_CM2_VHR4", "FGOALS_f3_H", "HiRAM_SIT_HR", "MRI_AGCM3_2_S", "EC_Earth3P_HR", "MPI_ESM1_2_XR", "NICAM16_8S"],
        "disable_bias_correction": True,
        "daily": ["temperature_2m_mean", "precipitation_sum"]
    }
    responses_climate = openmeteo.weather_api(climate_url, params=params_climate)
    response_climate = responses_climate[0]
    daily_climate = response_climate.Daily()
    
    temp_mean_models = np.array([daily_climate.Variables(i).ValuesAsNumpy()[0] for i in range(7)])
    precipitation_models = np.array([daily_climate.Variables(i + 7).ValuesAsNumpy()[0] for i in range(7)])
    
    temp_mean_avg = temp_mean_models.mean()
    precipitation_avg = precipitation_models.mean()
    
    # Weighted combination
    temp_weighted = 0.7 * ((temp_max + temp_min) / 2) + 0.3 * temp_mean_avg
    precipitation_weighted = 0.7 * precipitation + 0.3 * precipitation_avg
    
    return temp_weighted, precipitation_weighted

def predict_water_chemicals(geotiff_path, additional_features, lat, lon):
    wvr = extract_wvr_from_geotiff(geotiff_path)
    temp_weighted, precipitation_weighted = fetch_weather_data(lat, lon)
    input_features = np.array([wvr] + additional_features + [temp_weighted, precipitation_weighted]).reshape(1, -1)
    predictions = {}
    
    for target in targets:
        predictions[target] = ridge_models[target].predict(input_features)[0]
    
    return predictions

# Example Usage
geotiff_path = "example.tif"
additional_features = [10.5, 0.3, 0.8, 0.2]  # Example CHLA_RESULT, PIP_PT, TIP_PT, EVI values
latitude, longitude = 45.5, -122.6  # Example coordinates
predictions = predict_water_chemicals(geotiff_path, additional_features, latitude, longitude)
print(predictions)
